In [6]:
import os 
import tensorflow as tf
import numpy as np
import csv
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
# # make csv file with stupid order
Train='/opt/jupyterhub/SkinData/TrainingSet/'
Valid='/opt/jupyterhub/SkinData/ValidationSet/'
Test='/opt/jupyterhub/SkinData/TestSet/'
Train_csv='./fengkai/train_csv'
Valid_csv='./fengkai/valid_csv'
Test_csv='./fengkai/test_csv'
def batch_img_label(base_path,output_file):
    if 'Train' in base_path:
        img_1,img_2,img_3 = os.listdir(base_path)
    else:
        img_1,img_2,img_3,_ = os.listdir(base_path)
    print(img_1,img_2,img_3)
    with open (output_file, 'a') as csvfile:
        headers = ['image_id', 'Seborrheic', 'Nevus','Melanoma']
        writer = csv.DictWriter(csvfile, delimiter=',', lineterminator='\n',fieldnames=headers) 
        writer.writeheader()


        if not os.path.isfile(output_file):
            writer.writeheader()  # file doesn't exist yet, write a header


        for file_name in os.listdir(os.path.join(base_path,img_1)):
            writer.writerow({'image_id':os.path.join(os.path.join(base_path,img_1),file_name),'Seborrheic':1,'Nevus':0,'Melanoma':0})   
        for file_name in os.listdir(os.path.join(base_path,img_2)):
            writer.writerow({'image_id':os.path.join(os.path.join(base_path,img_2),file_name),'Seborrheic':0,'Nevus':1,'Melanoma':0})  
        for file_name in os.listdir(os.path.join(base_path,img_3)):
            writer.writerow({'image_id':os.path.join(os.path.join(base_path,img_3),file_name),'Seborrheic':0,'Nevus':0,'Melanoma':1})  

# batch_img_label(Train,Train_csv) 

In [8]:
# alexnet 

# check bn before relu...
def alexnet_full_connect(inputs,
               num_classes=3,
               is_training=True,
               dropout_keep_prob=0.5,
               spatial_squeeze=True,
               scope='alexnet_connect',
               global_pool=False):

    conv1=tf.layers.conv2d(inputs, 64, [11, 11], 4, padding='VALID')
    conv1=tf.layers.batch_normalization(conv1)
    pool1=tf.layers.max_pooling2d(conv1, [3, 3], 2)
    
    conv2=tf.layers.conv2d(pool1,192, [5, 5])
    conv2=tf.layers.batch_normalization(conv2)
    pool2=tf.layers.max_pooling2d(conv2,[3, 3], 2)
    
    conv3=tf.layers.conv2d(pool2,384, [3, 3])
    conv3=tf.layers.batch_normalization(conv3)
    conv4=tf.layers.conv2d(conv3, 384, [3, 3])
    conv4=tf.layers.batch_normalization(conv4)
    conv5=tf.layers.conv2d(conv4,256, [3, 3])
    conv5=tf.layers.batch_normalization(conv5)
    pool5=tf.layers.max_pooling2d(conv5,[3, 3], 2)
    
    flatten1=tf.contrib.layers.flatten(pool5)
    full_connect6=tf.contrib.layers.fully_connected(flatten1,num_outputs=4096)
    full_connect6=tf.layers.batch_normalization(full_connect6)
    drop_out1=tf.layers.dropout(full_connect6,dropout_keep_prob)
    full_connect7=tf.contrib.layers.fully_connected(drop_out1,num_outputs=4096)
    full_connect7=tf.layers.batch_normalization(full_connect7)
    drop_out2=tf.layers.dropout(full_connect7,dropout_keep_prob)
    full_connect8=tf.contrib.layers.fully_connected(drop_out2,num_outputs=num_classes)
    full_connect8=tf.layers.batch_normalization(full_connect8)
#     output=tf.nn.sigmoid(full_connect8)
    return full_connect8

In [9]:
def read_csv_batch(csv_file,batch_size,if_aug):
    
    shitname = tf.train.string_input_producer([csv_file],shuffle=True)
    reader = tf.TextLineReader(skip_header_lines=1)
    key, value = reader.read(shitname)

    value_shuffle=tf.random_shuffle(value)
    file_name,Seborrheic,Nevus,Melanoma = tf.decode_csv(value_shuffle,record_defaults=[[""],[0],[0],[0]])
    #one_hot_label
      #  file_name=tf.as_string(file_name).decode('ascii')
    one_hot_label=tf.stack([Seborrheic,Nevus,Melanoma])


    image = tf.train.string_input_producer([file_name],shuffle=False)

    img_reader = tf.WholeFileReader()
    #     print(input_queue[0])
    filename, b_img = img_reader.read(image)
    real_image=tf.image.decode_jpeg(b_img,channels=3)

    img = tf.cast(real_image, tf.float32)

    resized_image = tf.image.resize_image_with_crop_or_pad(img, 224, 224) 
    resized_image = tf.image.per_image_standardization(resized_image)
    if if_aug:
        resized_image = tf.image.random_flip_left_right(resized_image)
        
    #resized_image= tf.image.flip_left_right(resized_image)
    



    min_after_dequeue = 60
    capacity = min_after_dequeue + 1 * batch_size
    example_batch, label_batch = tf.train.shuffle_batch(
      [resized_image,one_hot_label], batch_size=batch_size, capacity=capacity,
      min_after_dequeue=min_after_dequeue) 
    return example_batch, label_batch

In [10]:
# evaluation
def evaluation(logits, labels):
    """Evaluate the quality of the logits at predicting the label.
    Args:
    logits: Logits tensor, float - [batch_size, NUM_CLASSES].
    labels: Labels tensor, int32 - [batch_size], with values in the
      range [0, NUM_CLASSES).
    Returns:
    A scalar int32 tensor with the number of examples (out of batch_size)
    that were predicted correctly.
    """
    with tf.variable_scope('accuracy') as scope:
        # to stablize the rank
        lbls= tf.argmax(labels, 1)
        correct = tf.nn.in_top_k(logits, lbls, 1)
        correct = tf.cast(correct, tf.float16)
        accuracy = tf.reduce_mean(correct)
        tf.summary.scalar(scope.name+'/accuracy', accuracy)
    return accuracy

In [ ]:
#enable gpu growth
logs_dir='./fengkai/alexnet_log/'
if not os.path.exists(logs_dir):
    os.mkdir(logs_dir)
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

result_log='./fengkai/result.txt'

if not os.path.exists(result_log):
    open(result_log, 'a').close()
with session as sess:
    # data training flow

    
    X = tf.placeholder(tf.float32, shape=[None, 224, 224, 3], name="image_shit")
    y = tf.placeholder(tf.int32, shape=[None, 3], name="label_shit")
    pred=alexnet_full_connect(X)
    batch_size=4


    # train op
    loss = tf.losses.softmax_cross_entropy(y, logits=pred)
    global_step = tf.train.get_or_create_global_step() #if it is default, learning rate should be 0.001

    learning_rate = tf.train.exponential_decay(0.001,global_step,10,0.98,staircase =True)
 
    optim=tf.train.MomentumOptimizer(learning_rate,0.9,use_nesterov=True).minimize(loss,global_step)
#momentum check 

###############
    validate_op=evaluation(pred, y)
    test_op=evaluation(pred,y)
    
    train_csv_tensor=tf.convert_to_tensor(Train_csv,dtype=tf.string)
    img_after_csv,lbl_after_csv=read_csv_batch(train_csv_tensor,batch_size,if_aug=True)
    
    valid_csv_tensor=tf.convert_to_tensor(Valid_csv,dtype=tf.string)
    valid_img,valid_label=read_csv_batch(valid_csv_tensor,batch_size,if_aug=False)
    
    test_csv_tensor=tf.convert_to_tensor(Test_csv,dtype=tf.string)
    test_img,test_label=read_csv_batch(test_csv_tensor,batch_size,if_aug=False)
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    sess.run(tf.global_variables_initializer())
    train_writer=tf.summary.FileWriter(logs_dir, sess.graph)
    
    saver=tf.train.Saver()
    
    loss_epoch=0
    
    print('{} start training {}'.format(15 *' #',15 *' #'))
    with open(result_log,'a') as result_file:
        
        for iter in range (0, 2000,batch_size):
            img_batch,lbl_batch=sess.run([img_after_csv,lbl_after_csv])
            #val_img_batch,val_lbl_batch=sess.run([valid_img,valid_label])
            _,train_loss=sess.run([optim,loss],feed_dict={X: img_batch,
                                   y: lbl_batch
                                   })
#             print('label values{}'.format(lbl_batch))
            #acc=sess.run([validate_op],feed_dict={X: val_img_batch, y: val_lbl_batch})            

            if iter % (20)==0:
                acc_epoch=0
                test_av_accuracy=0
                for val_i in range(0,150,batch_size):
                    val_img_batch,val_lbl_batch=sess.run([valid_img,valid_label])
                    acc=sess.run([validate_op],feed_dict={X: val_img_batch, y: val_lbl_batch})
                    acc_epoch+=acc[0]
                for t in range(0, 600,batch_size):
                    test_img_batch,test_lbl_batch=sess.run([test_img,test_label])
                    test_acc=sess.run([test_op],feed_dict={X: test_img_batch,
                                               y: test_lbl_batch
                                               }) 
                    test_av_accuracy+=test_acc[0]
        
                print('iter{},loss{},accuracy on validation{} on test{}'.format(iter,train_loss,acc_epoch/(150/batch_size),test_av_accuracy/(600/batch_size)) )
                
                result_file.write('iter{},loss{},accuracy on validation{}on test{}\n'.format(iter,train_loss,acc_epoch/(150/batch_size),test_av_accuracy/(600/batch_size)) )
                
                #print('the trainning loss and accuracy are {} and {}'.format(loss[0],acc[0]))
            if iter % (500) ==0:
                checkpoint_path = os.path.join(logs_dir, 'model.ckpt')
                saver.save(sess, checkpoint_path, global_step=iter)

        
    

 # # # # # # # # # # # # # # # start training  # # # # # # # # # # # # # # #
